# Idea
1. Pull sets of data from steam game
2. Get correct text from steam [text formatting](https://steamcommunity.com/comment/Recommendation/formattinghelp) from reviews forms
3. Remove rest of steam [text formatting](https://steamcommunity.com/comment/Recommendation/formattinghelp)
4. Clean from ascii art
5. Save non-english reviews to seperate db & remove from dataset
6. Author unravel
7. Word 2 number
8. Save to sqlite via duckdb

# Knowledge
29 sets of review_list from steam is about 0,5MB = 4 Mb

In [1]:
!python3 --version

Python 3.12.3


In [2]:
!pip install duckdb

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [1]:
import duckdb as db
import polars as pl
import requests
import sys

def find_game_by_name(game) -> tuple:
    str; url = "http://api.steampowered.com/ISteamApps/GetAppList/v0002/"
    dict; game_id_list = requests.get(url).json()

    if type(game) == int:
        identify = "appid"
    else:
        identify = "name"
        print("String input given")
    #print(game_id_list)
    for app in game_id_list["applist"]["apps"]:
        #print(app[identify])
        if app[identify] == game:
            return (app["appid"], app["name"])
    print("not found")
    return False, False

In [2]:
def calculate_memory(response,keys):
    """
    This is a tool which helps measurte objects' memory usage/size

    sys.getsizeof returns bytes(B)
    """
    mem = 0
    mem2 = 0
    for key1 in ['query_summary', 'reviews', 'cursor']:
        if response[key1].__class__ == dict:
            for key2 in response[key1].keys():
                if response[key1][key2].__class__ == dict:
                    for key3 in response[key1][key2].keys():
                        if response[key1][key2][key3].__class__ == dict:
                            print(f'({key1},{key2},{key3})')
                            continue
                        mem += sys.getsizeof(response[key1][key2][key3])
                    continue
                mem += sys.getsizeof(response[key1][key2])
            continue
        mem += sys.getsizeof(response[key1])
    for review in response['reviews']:
        for key in review['author'].keys():
            if review['author'][key].__class__ == dict:
                1+1
                #print(f'key: {key}, keys: {review["author"][key].keys()}')
            else:
                mem2 += sys.getsizeof(review['author'][key])
        for key in keys:
            if review[key].__class__ == dict:
                1+1
                #print(f'key: {key}, keys: {review[key].keys()}')
            else:
                mem2 += sys.getsizeof(review[key])
    return mem + mem2

In [3]:
parameters = {"json": 1, "filter":"recent", "num_per_page":100, "language": "english", "purchase_type":"all", "day_range":365, "cursor":"*"}

game = 'Hollow Knight'
if game.__class__ == str:
    game = find_game_by_name(game)[0]
print(game)
url = f'https://store.steampowered.com/appreviews/{game}'
response = requests.get(url, params=parameters).json()
del response['success']
response.keys()
keys = list(response['reviews'][0].keys())

String input given
367520


In [4]:
import math
iterations = math.ceil(response['query_summary']['total_reviews'] / 100) - 1
iterations

1317

In [5]:
#mem_list = [calculate_memory(response,keys)]
#for x in range(iterations):
#    print(x,flush=True,end='\r')
#    parameters['cursor'] = response['cursor']
#    response = requests.get(url, params=parameters).json()
#    del response['success']
#    mem_list.append(calculate_memory(response,keys))

allocated memory / ( max_size_100_set)

In [6]:
review_list_chunk_size = 2900

# Clean data

#### 1. Make test data
Need dataset that has english + other languages
#### 2. Build

In [7]:
# 1
# Making test dataset

def unravel_review_package(package):
    return [r"{}".format(x["review"]) for x in package["reviews"]]
parameters = {"json": 1, "filter":"recent", "num_per_page":100, "language": "english", "purchase_type":"all", "day_range":365, "cursor":"*"}
game = 'Hollow Knight'
if game.__class__ == str:
    game = find_game_by_name(game)[0]
print(game)
url = f'https://store.steampowered.com/appreviews/{game}'

formatting_texts = []

while False:
    response = requests.get(url, params=parameters).json()
    reviews = unravel_review_package(response)
    for review in reviews:
        if "[" in review:
            formatting_texts.append(review)
    if len(formatting_texts) > 25:
        break
    parameters["cursor"] = response["cursor"]
formatting_texts

String input given
367520


['It\'s Hollow Knight. It has >95% positive reviews (and most of those negative reviews are joke reviews anyways) and I\'m sure you\'ve heard just about all the praise there is for this game. So I\'m going to rant about what I don\'t like, even though I\'d still recommend it.\n\nHollow Knight is a decently fun game with incredibly fun movement and combat mechanics. \nUnfortunately, almost none of those mechanics ever get the chance to shine. I can only think of two bosses that have platforming as a part of the fight (maybe three, depending on if you count [spoiler]Radiance[/spoiler] and [spoiler]Absolute Radiance[/spoiler] as separate bosses) and even then it feels like an afterthought. Outside of that, any movement in boss fights can be boiled down to:\n"Oh, I\'m about to get hit. I should shade cloak through that." or,\n"Oh, the boss is running away. I should dash towards them."\n\nThere\'s also the issue of charms.\nOn the surface, you may think: "Wow, I get to customize a build to 

# Idea
Kerää about 200 table reviewiä niin tietää mitä niille sitten tehdä.

In [11]:
import pandas as pd
df = pd.read_csv("test_data_raw.csv", index_col=0)
with open("steam_format_example.txt", "r") as file:
    df = pd.concat([df, pd.DataFrame({"review":[file.read()]})])
df.reset_index(drop=True,inplace=True)
df

,review
0,It's Hollow Knight. It has >95% positive revie...
1,My review had to be trimmed to fit Steam's lim...
2,"Hollow Knight's graphics are really stunning, ..."
3,"Après avoir [spoiler]accepté le vide, qui étai..."
4,Hollow Knight is simply one of the most unique...
...,...
122,Many of you will be dead by the time the seque...
123,i beat this game on xbox and loved it. drop si...
124,epic
125,Good Game must play if u like metroidvanias!!


In [ ]:
df = pd.concat([df,pd.DataFrame()])
df.head()

In [32]:
for line in r.text.split("<"):
    if 'id="ReviewText"' in line:
        print(line)

div id="ReviewText">
								


In [ ]:
df = pd.concat([df,pd.DataFrame({"review":})])
df.head()

In [2]:
df[df.duplicated()]
int(12.9)
for x in range(4):
    print(x)
review_list = [(1,2),(3,4),(5,6)]
review_list[-1] = (review_list[-1][0],None)
review_list

0
1
2
3


[(1, 2), (3, 4), (5, None)]

In [22]:
import re
# For testing
temp = "[noparse] Doesn't parse [b]tags[/b] [/noparse] sieniä"
print(temp)
temp = re.sub(r'\[(/)?noparse\]', "",temp)
temp

[noparse] Doesn't parse [b]tags[/b] [/noparse] sieniä


" Doesn't parse [b]tags[/b]  sieniä"

In [3]:
import re
#https://realpython.com/intro-to-python-threading/
#https://stackoverflow.com/questions/6893968/how-to-get-the-return-value-from-a-thread

review_list = df.review.to_list()

new_list = list()
detections = {"Non_english": [],
              "Checkbox": {"start": "", "end": []},
              "Format": [],
              "Ascii_art": []}

def cleaner_func(start, end):
    global review_list
    text:str
    for text in review_list[start:end]:
        #new_list.append(text.split("\n\n"))
        # Checkbox listing pattern
        if "☐" in text and "☑" in text:
            detections["Checkbox"]["start"] = text
            temp = text.split("\n")
            #https://stackoverflow.com/questions/12091065/negative-pattern-matching-in-python
            temp = "\n".join([x.replace("☑ ", "").replace("\r","") for x in temp if "☐" not in x])
        temp = re.sub(r'\[(/)?[a-z](\d)?\]', "", temp)
        # Header "[h1]", "[/h1]"
        temp = re.sub(r'\[(/)?h\d+\]', "")
        # Bold
        temp = re.sub(r'\[(/)?b\]', "",temp)
        # Underlined
        temp = re.sub(r'\[(/)?u\]\s?', "",temp)
        # Italic
        temp = re.sub(r'\[(/)?i\]\s?', "",temp)
        # Crossed/strike
        temp = re.sub(r'\[(/)?strike\]', "",temp)
        # Hide/spoiler
        temp = re.sub(r'\\(\)?spoiler\]', "", temp)
        # Noparse (What does this do though?)
        temp = re.sub(r'\[(/)?noparse\]', "",temp)
        # Horizontal rule
        #TODO
        # Replace steam game's link with the game name
        #TODO
        # Purge links
        #TODO
        # Bullet list
        #TODO
        # Ordered list
        #TODO
        # Quotes
        #TODO
        # Purge code
        #TODO
        # Table
        #TODO
        # 7/10 game -> 7 out of 10
        temp = re.sub(r'\s\(\+\d+\)', "", temp)
        # 7 -> seven
        #TODO
        # Double spaces "  " -> " "
        detections["Checkbox"]["end"] = temp
        

In [4]:
import threading
def cleaner(num_thread = 1):
    """
    Input : Pandas (future Polars) dataframe
    Output: Cleaned dataframe
    """
    global review_list
    assert num_thread.__class__ == int
    if num_thread < 1:
        raise TypeError("Cannot have less than one (<1) threads")
    if num_thread != 1 and len(review_list) > 1:
        task_size = round(len(review_list) / num_thread)
        task_indexes = list()
        for x in range(num_thread):
            task_indexes.append((x*task_size,(x+1)*task_size))
        print(task_indexes)
        task_indexes[-1] = (task_indexes[-1][0],None)
    else:
        cleaner_func(0,None)
        return

    #parts = [(start,end) for part in range(len(review_list))]
    # IGNORE
    # First thread to be activated gets the tail end of data for that might have 1 more row of data
    # IGNORE

    threads = list()
    for x in range(num_thread):
        thread_obj = threading.Thread(target=cleaner_func, args=task_indexes[x])
        threads.append(thread_obj)
        thread_obj.start()
    for x in range(num_thread):
        threads[x].join()
    return


cleaner(num_thread=2)

[(0, 63), (63, 126)]


In [5]:
print(detections["Checkbox"]["end"])

Final Verdict: 98/100

Exceptional: 95-100 (95-100%)
Performance or gameplay significantly exceeds the passing mark.
Represents a superior level of quality, remarkable gameplay, and an outstanding gaming experience.
Highly Recommended -

Player Opinion Steam Review

Gameplay Experience (25/25)

Intuitive Controls - Easy to use.
Very Easy


Balanced Gameplay - Fair and equal.
Perfectly Balanced


Player Engagement - Involving and captivating.
Extremely


Decision Relevance - Meaningful choices.
Highly Meaningful


Rewarding Progression - Satisfying advancement.
Very Satisfying


Presentation & Immersion (23/25)

Compelling Storyline - Engaging narrative.
Extremely Interesting


Immersive Atmosphere - Captivating environment.
Completely Immersive


Visual Aesthetics - Stunning graphics.
Extremely Appealing


Audio Quality - High-quality sound.
Extremely Engaging


Polished HUD - Clean and functional interface.
Neutral


Design & Technical Aspects (25/25)

Game Design - Well-crafted mecha

In [6]:
#print(detections["Checkbox"]["end"])
text = detections["Checkbox"]["start"].split("\n")
temp = "\n".join([x.replace("☑ ", "").replace("\r","") for x in text if "☐" not in x])
temp = re.sub(r'\s\(\+\d+\)', "", temp)
temp = re.sub(r'\[(/)?[a-z](\d)?\]', "", temp)
print(temp)

Final Verdict: 98/100

Exceptional: 95-100 (95-100%)
Performance or gameplay significantly exceeds the passing mark.
Represents a superior level of quality, remarkable gameplay, and an outstanding gaming experience.
Highly Recommended -

Player Opinion Steam Review

Gameplay Experience (25/25)

Intuitive Controls - Easy to use.
Very Easy


Balanced Gameplay - Fair and equal.
Perfectly Balanced


Player Engagement - Involving and captivating.
Extremely


Decision Relevance - Meaningful choices.
Highly Meaningful


Rewarding Progression - Satisfying advancement.
Very Satisfying


Presentation & Immersion (23/25)

Compelling Storyline - Engaging narrative.
Extremely Interesting


Immersive Atmosphere - Captivating environment.
Completely Immersive


Visual Aesthetics - Stunning graphics.
Extremely Appealing


Audio Quality - High-quality sound.
Extremely Engaging


Polished HUD - Clean and functional interface.
Neutral


Design & Technical Aspects (25/25)

Game Design - Well-crafted mecha